In [2]:
from ultralytics import YOLO
import cv2
import os
from datetime import datetime

# Load trained model
model = YOLO('runs/detect/fan_detector2/weights/best.pt')

# Open video feed
video_path = 'sample_goal_clip.mp4'  
cap = cv2.VideoCapture(video_path)

# Create output directory for alerts
os.makedirs('alerts', exist_ok=True)


In [3]:
def get_team_from_class(class_id):
    return "hilal" if class_id in [0, 1, 2, 3] else "ittihad"

def is_cheering(class_id):
    return class_id in [1, 5]  # cheering classes for both teams


In [4]:
def detect_crowd_reaction(results):
    hilal_cheer = 0
    ittihad_cheer = 0
    all_fans = []

    for r in results:
        for box in r.boxes:
            class_id = int(box.cls)
            conf = float(box.conf)
            if conf < 0.5: continue

            team = get_team_from_class(class_id)
            cheering = is_cheering(class_id)

            if team == "hilal" and cheering:
                hilal_cheer += 1
            elif team == "ittihad" and cheering:
                ittihad_cheer += 1

            fan_data = {
                'box': box.xyxy[0].cpu().numpy().astype(int),
                'team': team,
                'cheering': cheering,
                'class_id': class_id
            }
            all_fans.append(fan_data)

    return hilal_cheer, ittihad_cheer, all_fans


In [5]:
def is_misplaced(fan, crowd_team, crowd_ratio=0.8):
    return fan['team'] != crowd_team and not fan['cheering']


In [6]:
frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    results = model(frame)
    hilal_cheer, ittihad_cheer, fans = detect_crowd_reaction(results)

    # Determine crowd mood
    if hilal_cheer > ittihad_cheer * 1.5:
        crowd_team = "hilal"
    elif ittihad_cheer > hilal_cheer * 1.5:
        crowd_team = "ittihad"
    else:
        crowd_team = None  # neutral

    for fan in fans:
        x1, y1, x2, y2 = fan['box']
        color = (0, 255, 0)

        if crowd_team and is_misplaced(fan, crowd_team):
            color = (0, 0, 255)  # red box
            cv2.imwrite(f"alerts/frame_{frame_id}_{datetime.now().strftime('%H%M%S')}.jpg", frame)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"{fan['team']} - {'cheering' if fan['cheering'] else 'idle'}"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow('Security Feed', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_id += 1

cap.release()
cv2.destroyAllWindows()



0: 320x416 1 hilal_cheering, 4 ittihad_sittings, 120.8ms
Speed: 9.9ms preprocess, 120.8ms inference, 14.0ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 2 hilal_sittings, 1 hilal_cheering, 1 ittihad_sitting, 86.5ms
Speed: 2.0ms preprocess, 86.5ms inference, 3.6ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 2 hilal_sittings, 1 hilal_cheering, 2 ittihad_sittings, 62.7ms
Speed: 3.0ms preprocess, 62.7ms inference, 4.1ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 3 hilal_sittings, 1 hilal_cheering, 2 ittihad_sittings, 78.0ms
Speed: 4.4ms preprocess, 78.0ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 1 hilal_sitting, 1 hilal_cheering, 3 ittihad_sittings, 69.1ms
Speed: 4.7ms preprocess, 69.1ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 3 hilal_sittings, 1 hilal_cheering, 1 ittihad_sitting, 67.2ms
Speed: 2.1ms preprocess, 67.2ms inference, 1.0ms postprocess per image at shape (1,